#### Point Map Visualization

Point map visualization is natively supported, if your table outputs longitude-latitude pairs. It is recommended to limit the query to not too many rows for performance reasons.

In [0]:
%pip install duckdb --quiet

import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType, BinaryType, StringType # 

def st_duckdb_x_func(a: pd.Series) -> pd.Series:
    try:
        import duckdb
    except ModuleNotFoundError as mfe:
        raise Exception(
            "'duckdb' not installed: run '%pip install duckdb' first."
        ) from mfe

    duckdb.sql("SET extension_directory = '/tmp/duckdb_ext'")
    try:
        duckdb.load_extension("spatial")
    except duckdb.duckdb.IOException:
        duckdb.install_extension("spatial")
        duckdb.load_extension("spatial")

    df = pd.DataFrame({"a": a})  # noqa: F841

    # TODO: rewrite with te Expression API (a la lonboard)
    res = duckdb.sql(
        """
select
    st_x(st_geomfromwkb(a))::string res
from df"""
    ).df()["res"]

    return res


st_duckdb_x = F.pandas_udf(st_duckdb_x_func, returnType=StringType()) # DoubleType())
spark.udf.register("st_duckdb_x", st_duckdb_x)

def st_duckdb_y_func(a: pd.Series) -> pd.Series:
    try:
        import duckdb
    except ModuleNotFoundError as mfe:
        raise Exception(
            "'duckdb' not installed: run '%pip install duckdb' first."
        ) from mfe

    duckdb.sql("SET extension_directory = '/tmp/duckdb_ext'")
    try:
        duckdb.load_extension("spatial")
    except duckdb.duckdb.IOException:
        duckdb.install_extension("spatial")
        duckdb.load_extension("spatial")
        
    df = pd.DataFrame({"a": a})  # noqa: F841

    # TODO: rewrite with te Expression API (a la lonboard)
    res = duckdb.sql(
        """
select
    st_y(st_geomfromwkb(a)) res
from df"""
    ).df()["res"]

    return res


st_duckdb_y = F.pandas_udf(st_duckdb_y_func, returnType=DoubleType())
spark.udf.register("st_duckdb_y", st_duckdb_y)


In [0]:
spark.read.parquet("s3://overturemaps-us-west-2/release/2025-05-21.0/theme=places/type=place").createOrReplaceTempView("place")

In [0]:
%sql
select
  *,
  st_duckdb_x(geometry) lon,
  st_duckdb_y(geometry) lat
from
  place
where
  categories.primary = "restaurant"
limit 10000
-- NOTE: you'll see a bunch of data errors, i.e. lots of restaurants in the Antarctica, but that's data error in the source data.

![point map](./img/point_map.png)